In [ ]:
!pip3 install scikit-image
!pip3 install nose
!pip3 install tornado
!pip3 install keras
!pip3 install keras_vggface
!pip3 install pandas
!pip3 install h5py
!pip3 install keras_applications

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.python.lib.io import file_io
from tensorflow.keras import utils
from skimage.transform import resize
from keras import backend as K
from keras import utils as np_utils
from keras_vggface.vggface import VGGFace
from keras.models import Model
from keras.layers import Flatten, Dense 
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, SGD
from keras.callbacks import TensorBoard, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping, Callback, ModelCheckpoint
import h5py

In [ ]:
folder = './'

img_height, img_width = 197, 197

num_classes         = 7   # ['Anger', 'Disgust', 'Fear', 'Happiness', 'Sadness', 'Surprise', 'Neutral']
epochs_top_layers   = 5
epochs_all_layers   = 100
batch_size          = 128

train_dataset	= 'train.csv'
eval_dataset 	= 'test.csv'

In [ ]:
base_model = VGGFace(
    model       = 'resnet50',
    include_top = False,
    weights     = 'vggface',
    input_shape = (img_height, img_width, 3))

x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation = 'relu')(x)

predictions = Dense(num_classes, activation = 'softmax')(x)

model = Model(inputs = base_model.input, outputs = predictions)

In [ ]:
def preprocess_input(x):
    x -= 128.8006 # np.mean(train_dataset)
    return x

def get_data(dataset):
    file_stream = file_io.FileIO(dataset, mode='r')
    data = pd.read_csv(file_stream)
    pixels = data['pixels'].tolist()
    images = np.empty((len(data), img_height, img_width, 3))
    i = 0

    for pixel_sequence in pixels:
        single_image = [float(pixel) for pixel in pixel_sequence.split(' ')]  # Extraction of each single
        single_image = np.asarray(single_image).reshape(48, 48) # Dimension: 48x48
        single_image = resize(single_image, (img_height, img_width), order = 3, mode = 'constant') # Dimension: 139x139x3
        ret = np.empty((img_height, img_width, 3))  
        ret[:, :, 0] = single_image
        ret[:, :, 1] = single_image
        ret[:, :, 2] = single_image
        images[i, :, :, :] = ret
        i += 1
    
    images = preprocess_input(images)
    labels = utils.to_categorical(data['emotion'])

    return images, labels    

train_data_x, train_data_y  = get_data(train_dataset)
val_data  = get_data(eval_dataset)

train_datagen = ImageDataGenerator(
    rotation_range  = 10,
    shear_range     = 10, # 10 degrees
    zoom_range      = 0.1,
    fill_mode       = 'reflect',
    horizontal_flip = True)

train_generator = train_datagen.flow(
    train_data_x,
    train_data_y,
    batch_size  = batch_size)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

model.compile(
    optimizer   = Adam(lr = 1e-3, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, decay = 0.0), 
    loss        = 'categorical_crossentropy', 
    metrics     = ['accuracy'])

model.fit_generator(
    generator           = train_generator,
    steps_per_epoch     = len(train_data_x) // batch_size,  # samples_per_epoch / batch_size
    epochs              = epochs_top_layers,                            
    validation_data     = val_data)

In [ ]:
for layer in model.layers:
    layer.trainable = True

model.compile(
    optimizer=SGD(lr=1e-4, momentum=0.9, decay=0.0, nesterov=True),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

def scheduler(epoch):
    updated_lr = K.get_value(model.optimizer.lr) * 0.5
    if (epoch % 3 == 0) and (epoch != 0):
        K.set_value(model.optimizer.lr, updated_lr)
        print(K.get_value(model.optimizer.lr))
    return K.get_value(model.optimizer.lr)

reduce_lr = LearningRateScheduler(scheduler)

reduce_lr_plateau = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    mode='auto',
    min_lr=1e-8)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    mode='auto')

check_point = ModelCheckpoint("model_weights.h5",
                              monitor='val_loss',
                              verbose=1,
                              save_best_only=True,
                              mode='auto')

history = model.fit_generator(
    generator=train_generator,
    steps_per_epoch=len(train_data_x) // batch_size,
    epochs=epochs_all_layers,
    validation_data=val_data,
    callbacks=[reduce_lr, reduce_lr_plateau, early_stop, check_point])

model.save(folder + 'ResNet-50.h5')